In [142]:
import requests
import urllib.parse
import openrouteservice
from openrouteservice import convert
import folium
import json
import time
import pandas as pd
import branca
import numpy as np
import math
import sys
sys.path.append('../utils/')
from utils import *
du = DB_Utils()
client = openrouteservice.Client(key='5b3ce3597851110001cf6248eba38563b623444f90c5626fa0f0b16d')
colors = ['red', 'blue', 'green', 'purple', 'orange', 'darkred', 'lightred', 'beige', 'darkblue', 'darkgreen', 'cadetblue', 'darkpurple', 'white', 'pink', 'lightblue', 'lightgreen', 'gray', 'black', 'lightgray']
state_geo = "https://raw.githubusercontent.com/python-visualization/folium/master/examples/data/us-states.json"

def distance_ecli(*arg):
    x1,x2,y1,y2 = arg
    return math.hypot(x2-x1, y2-y1)

def get_lon_lat(orginal_address):
    url = 'https://nominatim.openstreetmap.org/search/' + urllib.parse.quote(orginal_address) + ', USA' +'?format=json'
    response = requests.get(url).json()
    lat = float(response[0]["lat"])
    lon = float(response[0]["lon"])
    print(orginal_address, lon, lat)
    return lon, lat

def get_drive_distance(coords):
    time.sleep(1)
    res = client.directions(coords)
    geometry = client.directions(coords)['routes'][0]['geometry']
    decoded = convert.decode_polyline(geometry)
    distance_km = round(res['routes'][0]['summary']['distance']/1000, 1)
    leadtime_hr = round(res['routes'][0]['summary']['duration']/60,1)
    return distance_km, leadtime_hr, decoded

# data : states 별 onhand, order list, inbound 정보
sql = '''
SELECT t1.StatesCode AS State, SUM(ON_HAND) AS PCS, Latitude, Longtitude  FROM (
	select StatesCode, ON_HAND  from US_STATES_CODE usc 
	LEFT JOIN (select LEFT(WH_Location,2) AS WH_Location, CONVERT(INT, ON_HAND) AS ON_HAND  from ATP_BI_ENDING_ONHAND abeo 
	where Batch_Date ='2022-9-19' and Item_Code ='30390156H' and Dates = '2022-09-20'
	) AS abeo ON usc.StatesCode = abeo.WH_Location
	
	UNION ALL 
	
	select 
		LEFT(WH_Location,2) AS StatesCode
		,SUM(Intake) AS ON_HAND
	from ATP_BI_ENDING_ONHAND abeo 
	where Batch_Date ='2022-9-19' and Item_Code ='30390156H' and Dates >= '2022-09-20' and Dates  < '2022-09-24'
	GROup by LEFT(WH_Location,2)
	) as t1

	LEFT JOIN (
		select 
			StatesCode
			, AVG(Latitude) AS Latitude
			, AVG(Longtitude) AS Longtitude 
		from US_CITY_CODE ucc 
		GROUP BY StatesCode ) AS ucc 
	ON t1.StatesCode  = ucc.StatesCode 
	
	GROUP BY t1.StatesCode, Latitude, Longtitude
	ORDER BY SUM(ON_HAND) desc
'''

sql2 = '''
    select CustomerPO_Num , City, LEFT(WH, 2) AS WH , CONVERT(INT, SUM(PCS)) AS PCS, cc.Friday AS ETD from ORDER_STATUS_REPORT osr 
    LEFT JOIN CW_CALENDER cc ON format(CONVERT(INT, WeekNum), '00') = format(CONVERT(INT, REPLACE(osr.ALLOC_CW, 'CW' , '')), '00') AND (cc.Year = osr.year)
    where osr.Updated_Date ='2022-09-16' and Item_Code ='30390156H' and cc.Friday >='2022-09-19' and cc.Friday  < '2022-09-24'
    GROUP BY CustomerPO_Num , City, LEFT(WH, 2), cc.Friday
    ORDER BY cc.Friday, SUM(PCS) desc
'''

sql3 = '''
    select 
        POD,Item_Code 
        , WH_ETA 
        , MAX(ucc.Latitude)  AS Latitude
        , MAX(ucc.Longtitude) AS Longtitude
        , SUM(PCS) AS PCS 
    from ATP_INBOUND_OVERSEA_HISTORY aid 
    LEFT JOIN (
        select 
            StatesCode
            , AVG(Latitude) AS Latitude
            , AVG(Longtitude) AS Longtitude 
        from US_CITY_CODE ucc 
        GROUP BY StatesCode ) AS ucc 
    ON LEFT(aid.WH_Name,2)  = ucc.StatesCode 
    where Batch_Date ='2022-09-19' and Item_Code ='30390156H' 
    and WH_ETA  >= '2022-09-20' and WH_ETA  <= '2022-09-24'
    GROUP BY POD, Item_Code , WH_ETA 
'''
# state_data = pd.read_csv('./state_onhand.csv')
# df = pd.read_csv('../dev/outbound.csv').sort_values(by = 'pcs', ascending = False).reset_index(drop = True)
# df['dist'] = df[['Latitude' , 'C_Latitude','Longtitude', 'C_Longtitude']].apply(lambda x:distance_ecli(*x), axis = 1)

In [143]:
du.connect_azuredb()
state_data = du.fetch_data(sql)
du.connect_azuredb()
df = du.fetch_data(sql2)
df['C_longtitude'] , df['C_latitude']= zip(*df['City'].apply(lambda x:get_lon_lat(x)))
state_data_gp = pd.merge(state_data,
        df.groupby('WH').agg({'PCS' : 'sum'}).reset_index(),
        left_on = ['State'],
        right_on = ['WH'],
        how = 'left')
state_data_gp['PCS_y'].fillna(0, inplace = True)
state_data_gp['Ending_Onhand'] = state_data_gp['PCS_x'] - state_data_gp['PCS_y']
state_data = state_data_gp[['State', 'PCS_x', 'Ending_Onhand', 'Latitude', 'Longtitude']]
state_data = state_data.rename(columns = {'PCS_x'  : 'PCS'})

 Jacksonville  -81.655651 30.3321838
Riverside -117.3961623 33.9533546
Phoenix -112.074141 33.4484367
 Orlando  -81.3790304 28.5421109
Glen Rock -74.1329208 40.9628758
Sacramento -121.493895 38.5810606
 Orlando  -81.3790304 28.5421109
San Leandro -122.1560768 37.7249296
Newark -74.1723667 40.735657
Bakersfield -119.0194639 35.3738712
Chatsworth -96.5159408 42.9156667
Chatsworth -96.5159408 42.9156667
Katy -95.8243956 29.7857853
Las Vegas -115.148516 36.1672559
Phoenix -112.074141 33.4484367
Anaheim -117.911732 33.8347516
Denver -104.984862 39.7392364
El Paso  -106.441207 31.7865623
Albany -73.754968 42.6511674
Austin -97.7436995 30.2711286
Milwaukie -122.6412523 45.4440051
Dallas -96.7968559 32.7762719
Bakersfield -119.0194639 35.3738712
San Diego -117.1627728 32.7174202
Ontario -117.6484304 34.065846
Sacramento -121.493895 38.5810606
Weslaco -97.9907362 26.1594305
Riverside -117.3961623 33.9533546
Fresno -119.7848307 36.7394421
Garland -96.6388833 32.912624
Houston -95.3676974 29.7589

In [144]:
du.connect_azuredb()
inbound_data = du.fetch_data(sql3)

# Map initialization and divide by state's names

In [145]:
m = folium.Map(location=[32.960870, -83.571119],zoom_start=5, control_scale=True,tiles="cartodbpositron")
cp = folium.Choropleth(
    geo_data=state_geo,
    data=state_data,
    columns=["State", 'PCS', 'Ending_Onhand'],
    key_on="feature.id",
    fill_color="YlGn",
    fill_opacity=0.8,
    line_opacity=0.1,
    nan_fill_opacity = 0.3,
    legend_name="Onhand Qty(PCS)").add_to(m)
state_data_indexed = state_data.set_index('State')

for s in cp.geojson.data['features']:
    s['properties']['PCS'] = np.float64(state_data_indexed.loc[s['id'], 'PCS'])
    s['properties']['Ending_Onhand'] = np.float64(state_data_indexed.loc[s['id'], 'Ending_Onhand'])

    folium.GeoJsonTooltip(['name', 'PCS', 'Ending_Onhand']).add_to(cp.geojson)
m.save('map.html')

# Marking each Customer PO's & address sites

In [146]:
for idx in range(len(df)):
    print(df.iloc[idx]['C_longtitude'],df.iloc[idx]['C_latitude'])
    folium.Marker(
                    location=list((df.iloc[idx]['C_latitude'], df.iloc[idx]['C_longtitude'])), #delivery_coordinates
                    popup="{}".format(df['CustomerPO_Num'].iloc[idx]),
                    icon=folium.Icon(color='green', opacity = 0.5),
                    tooltip = df.iloc[idx]['CustomerPO_Num'] + ' & ' + df.iloc[idx]['PCS'].astype('str') ).add_to(m)
m.save('map.html')

-81.655651 30.3321838
-117.3961623 33.9533546
-112.074141 33.4484367
-81.3790304 28.5421109
-74.1329208 40.9628758
-121.493895 38.5810606
-81.3790304 28.5421109
-122.1560768 37.7249296
-74.1723667 40.735657
-119.0194639 35.3738712
-96.5159408 42.9156667
-96.5159408 42.9156667
-95.8243956 29.7857853
-115.148516 36.1672559
-112.074141 33.4484367
-117.911732 33.8347516
-104.984862 39.7392364
-106.441207 31.7865623
-73.754968 42.6511674
-97.7436995 30.2711286
-122.6412523 45.4440051
-96.7968559 32.7762719
-119.0194639 35.3738712
-117.1627728 32.7174202
-117.6484304 34.065846
-121.493895 38.5810606
-97.9907362 26.1594305
-117.3961623 33.9533546
-119.7848307 36.7394421
-96.6388833 32.912624
-95.3676974 29.7589382
-117.6484304 34.065846
-97.1071186 32.7355816
-97.1071186 32.7355816


# Navigating the driving route from warehouse -> customers

In [147]:
total_km = 0
total_hr = 0
for idx in range(len(df)):
    try:
        print(df['CustomerPO_Num'].iloc[idx])
        Longtitude = state_data[state_data['State'] == df.iloc[idx]['WH']]['Latitude'].values[0]
        Latitude = state_data[state_data['State'] == df.iloc[idx]['WH']]['Longtitude'].values[0]
        coords = ((Latitude, Longtitude),(df['C_longtitude'].iloc[idx], df['C_latitude'].iloc[idx]))
        distance_km , leadtime_hr, decoded = get_drive_distance(coords)
        folium.PolyLine(
                    locations=[[i[1],i[0]] for i in decoded['coordinates']], 
                    color = 'blue',
                    weight = round(df.PCS.iloc[idx] / df.PCS.max(),2) * 20,
                    opacity = 0.3,
                    tooltip = "{}km {}hr {}pcs".format(distance_km , leadtime_hr, df.PCS.iloc[idx])
        ).add_to(m)
        total_hr = total_km + leadtime_hr
        total_km = total_km + distance_km
    except:
        pass
#     if idx == 10:
#         break
        
m.save('map.html')

2022-08_PT2
6435-JH991006S_PT3
8756-KC997099S_PT4
5621-CH906081S_PT7
114522_PT3
6552-NZ970747S_PT4
5621-CH906081S_PT7.1
5040-JB981599S_PT1
7492-JM907557S
0311-TC912922S_PT4
1024-BO942405S_PT5
1024-BO942405S_PT6
3273-RK900267S_PT3
4090-RT906297S_PT6
7539-712326
1521-BL967482S_PT1
1875-DN918077S_PT10


c:\users\qcells\appdata\local\programs\python\python36\lib\site-packages\openrouteservice\client.py:214: UserWarning: Rate limit exceeded. Retrying for the 1st time.
  stacklevel=1)
c:\users\qcells\appdata\local\programs\python\python36\lib\site-packages\openrouteservice\client.py:214: UserWarning: Rate limit exceeded. Retrying for the 2nd time.
  stacklevel=1)


1987-SS904067S_PT5
0120-WL904242S_PT8


c:\users\qcells\appdata\local\programs\python\python36\lib\site-packages\openrouteservice\client.py:214: UserWarning: Rate limit exceeded. Retrying for the 3rd time.
  stacklevel=1)
c:\users\qcells\appdata\local\programs\python\python36\lib\site-packages\openrouteservice\client.py:214: UserWarning: Rate limit exceeded. Retrying for the 4th time.
  stacklevel=1)
c:\users\qcells\appdata\local\programs\python\python36\lib\site-packages\openrouteservice\client.py:214: UserWarning: Rate limit exceeded. Retrying for the 5th time.
  stacklevel=1)
c:\users\qcells\appdata\local\programs\python\python36\lib\site-packages\openrouteservice\client.py:214: UserWarning: Rate limit exceeded. Retrying for the 6th time.
  stacklevel=1)


0236-LUMIO921128S_PT4
5971-CC908662S_PT3
BPO092022-400H
BPS0922-3-400A
BPS0922-3-400B
BPS0922-3-400C
BPS0922-3-400D
PO4199_PT3
PO7042_PT6
PO7043_PT3
PO7046_PT5
PO7048_PT4
HQCV3CA3494_PT2.1
0237-KI907032S_PT1
0237-KI907084S_PT9


# Inbound information

In [149]:
for idx in range(len(inbound_data)):
    iframe = folium.IFrame(width=300, height=300)
    
    folium.Marker(
        location=list((inbound_data.iloc[idx]['Latitude'], inbound_data.iloc[idx]['Longtitude'])),
        icon= folium.Icon(color='red', icon='star'),
        tooltip = "INBOUND QTY : {}pcs ".format(inbound_data.iloc[idx]['PCS'])
    ).add_to(m)
m.save('map.html')

In [ ]:
folium.LayerControl().add_to(m)
m.save('map.html')

# The closest warehouse algorithm

In [152]:
state_data.dropna()

,State,PCS,Ending_Onhand,Latitude,Longtitude
0,CA,37874.0,13330.0,36.897181,-120.192245
1,WA,32832.0,30528.0,47.342100,-121.371094
2,NJ,4512.0,-4928.0,40.312849,-74.519086
3,TX,3840.0,3840.0,30.752906,-97.778658
4,GA,0.0,0.0,32.960870,-83.571119


In [155]:
df.head()

,CustomerPO_Num,City,WH,PCS,ETD,C_longtitude,C_latitude
0,2022-08_PT2,Jacksonville,NJ,5344,2022-09-23,-81.655651,30.332184
1,6435-JH991006S_PT3,Riverside,CA,2304,2022-09-23,-117.396162,33.953355
2,8756-KC997099S_PT4,Phoenix,CA,2304,2022-09-23,-112.074141,33.448437
3,5621-CH906081S_PT7,Orlando,NJ,1792,2022-09-23,-81.379030,28.542111
4,114522_PT3,Glen Rock,CA,1664,2022-09-23,-74.132921,40.962876


In [186]:
for i in [df.iloc[idx][['CustomerPO_Num', 'C_latitude', 'C_longtitude']].tolist() for idx in range(len(df))]:
    cpo_coord = i[1:]
    dist = []
    for idx in range(len(state_data.dropna())):
        wh_coord = state_data.dropna().iloc[idx][['Latitude' ,'Longtitude']].tolist()
        dist.append(distance_ecli(cpo_coord[0], wh_coord[0], cpo_coord[1], wh_coord[1]))
    print(i, state_data['State'].iloc[np.argmin(dist)])

['2022-08_PT2', 30.3321838, -81.655651] GA
['6435-JH991006S_PT3', 33.9533546, -117.3961623] CA
['8756-KC997099S_PT4', 33.4484367, -112.074141] CA
['5621-CH906081S_PT7', 28.5421109, -81.3790304] GA
['114522_PT3', 40.9628758, -74.1329208] NJ
['6552-NZ970747S_PT4', 38.5810606, -121.493895] CA
['5621-CH906081S_PT7.1', 28.5421109, -81.3790304] GA
['5040-JB981599S_PT1', 37.7249296, -122.1560768] CA
['7492-JM907557S', 40.735657, -74.1723667] NJ
['0311-TC912922S_PT4', 35.3738712, -119.0194639] CA
['1024-BO942405S_PT5', 42.9156667, -96.5159408] TX
['1024-BO942405S_PT6', 42.9156667, -96.5159408] TX
['3273-RK900267S_PT3', 29.7857853, -95.8243956] TX
['4090-RT906297S_PT6', 36.1672559, -115.148516] CA
['7539-712326', 33.4484367, -112.074141] CA
['1521-BL967482S_PT1', 33.8347516, -117.911732] CA
['1875-DN918077S_PT10', 39.7392364, -104.984862] TX
['1987-SS904067S_PT5', 31.7865623, -106.441207] TX
['0120-WL904242S_PT8', 42.6511674, -73.754968] NJ
['0236-LUMIO921128S_PT4', 30.2711286, -97.7436995] TX


In [39]:
# recommandation for new warehouse deu to shortage of stock, better route 
    # check the closest warehouse from each CPO
    # check inbound POD qty. if possible, change into POD -> new Warehouse
    # check how much warehouse need more (or check if ending onhand negative qty)
    # check transfer would be helpful 
    # check just itemcode change would be better

# total km, hr applied recommandations. 


In [16]:
for idx in range(len(df)):
    print(df['CustomerPO_Num'].iloc[idx])
    coords = ((df['Longtitude'].iloc[idx], df['Latitude'].iloc[idx]),(df['C_Longtitude'].iloc[idx], df['C_Latitude'].iloc[idx]))
    distance_km , leadtime_hr, decoded = get_drive_distance(coords)
    folium.PolyLine(
                locations=[[i[1],i[0]] for i in decoded['coordinates']], 
                color = colors[idx],
                weight = round(df.pcs.iloc[idx] / df.pcs.max(),2) * 15,
                opacity = 0.5,
                tooltip = "{} / {}pcs".format(df['Customer_Address'].iloc[idx], df.pcs.iloc[idx])).add_to(m)
    
#     folium.Marker(
#         location=list(coords[0][::-1]), #wh_coord_coordinates
#         popup=df.WH_Location.iloc[idx], 
#         icon=folium.Icon(icon = 'glyphicon glyphicon-cloud' , color= colors[idx], opacity = 0.5),
#         tooltip = 'WH').add_to(m)

    folium.Marker(
        location=list(coords[1][::-1]), #delivery_coordinates
        popup="{} / {}km / {}hrs".format(df['Customer_Address'].iloc[idx], distance_km, leadtime_hr),
        icon=folium.Icon(color=colors[idx], opacity = 0.5),
        tooltip = 'Customer').add_to(m)
    if idx == 3:
        break
folium.LayerControl().add_to(m)
m.save('map.html')

3443 E. Central Ave.
2825 E. Jurupa St
380 South Worcester St
31353 Huntwood Ave


In [47]:
df_order = df[['Customer_Address', 'pcs', 'C_Latitude', 'C_Longtitude']].sort_values(by = 'pcs', ascending = False).reset_index(drop = True)
df_wh = df[['WH_Location', 'Latitude', 'Longtitude']].groupby('WH_Location').agg({'Latitude' : 'mean', 'Longtitude' : 'mean'}).reset_index()

In [54]:
for idx in range(len(df_order)):
    min_dist_wh = 1000
    addr = df_order.iloc[idx]['Customer_Address']
    pcs = df_order.iloc[idx]['pcs']
    x1 = df_order.iloc[idx]['C_Latitude']
    y1 = df_order.iloc[idx]['C_Longtitude']
    
    for idx2 in range(len(df_wh)):
        wh = df_wh.iloc[idx2]['WH_Location']
        x2 = df_wh.iloc[idx2]['Latitude']
        y2 = df_wh.iloc[idx2]['Longtitude']
        dist = distance_ecli(x1, x2, y1, y2)
        if min_dist_wh > dist:
            min_dist_wh = dist 
            onhand_pcs = state_data[state_data['State'] == wh]['Onhand'].values[0]            
            if onhand_pcs >= pcs:
                print("THIS IS THE SHORTEST WAY -> " + addr + ' to ' + wh + ' : ' + str(round(dist,1)))
                print('OK : OnHAND PCS Check : ' + str(onhand_pcs) + 'pcs')
            else:
                print("THIS IS THE SHORTEST WAY -> " + addr + ' to ' + wh + ' : ' + str(round(dist,1)))
                print('NO : OnHAND PCS Check : ' + str(onhand_pcs) + 'pcs')
    print('\n')
    if idx == 10:
        break

THIS IS THE SHORTEST WAY -> 3443 E. Central Ave. to CA : 22.9
OK : OnHAND PCS Check : 53770pcs
THIS IS THE SHORTEST WAY -> 3443 E. Central Ave. to GA : 14.5
NO : OnHAND PCS Check : 1268pcs


THIS IS THE SHORTEST WAY -> 2825 E. Jurupa St to CA : 3.9
OK : OnHAND PCS Check : 53770pcs


THIS IS THE SHORTEST WAY -> 380 South Worcester St to CA : 49.3
OK : OnHAND PCS Check : 53770pcs
THIS IS THE SHORTEST WAY -> 380 South Worcester St to GA : 15.3
NO : OnHAND PCS Check : 1268pcs
THIS IS THE SHORTEST WAY -> 380 South Worcester St to NJ : 3.7
OK : OnHAND PCS Check : 50084pcs


THIS IS THE SHORTEST WAY -> 31353 Huntwood Ave to CA : 2.0
OK : OnHAND PCS Check : 53770pcs


THIS IS THE SHORTEST WAY -> 12085 Highland Road to CA : 30.5
OK : OnHAND PCS Check : 53770pcs
THIS IS THE SHORTEST WAY -> 12085 Highland Road to GA : 8.4
NO : OnHAND PCS Check : 1268pcs


THIS IS THE SHORTEST WAY -> 4601 Spring Hill Road to CA : 44.3
OK : OnHAND PCS Check : 53770pcs
THIS IS THE SHORTEST WAY -> 4601 Spring Hill Ro